In [ ]:
from gurobipy import *
import time
import pandas as pd
from scipy.spatial.distance import euclidean
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
path = 'customers.txt'
# read text file
with open(path, encoding='utf8') as f:
    lines = f.readlines()
    
rows = []
for line in lines:
    row = line.strip().split('\t')
    rows.append(row)
df = pd.DataFrame(rows, columns = rows[0])
df = df.drop(0)
df = df.set_index('CUST NO.')

first_row = df.iloc[0]
df = pd.concat([df, first_row.to_frame().transpose()], ignore_index=True)
df = df.astype({'X COORD.': 'float', 'Y COORD.': 'float'})#, 'POPULATION NO.': 'int', 'SURVIVAL_PROBABILITY': 'float'})
df.head()

In [ ]:
Ndrones = 2 # Número de drones
k = Ndrones
n = len(df) - 2 # Número de clientes


N = range(1, len(df.index) - 1) # Índice de clientes {1, 2, ..., n}
V = range(len(df.index)) # Índice de clientes y depósito {0, 1, ..., n+1}
S = range(1,Ndrones + 1) # Índice de rutas inicales 


# Se crean los arcos del grafo
arcos = []
for i in V:
    for j in V:
        if i != j:
            if i != len(V) - 1 and j != 0:
                arcos.append(tuple((i,j)))
arcos.remove((0, len(df) - 1))



# Se calculan los costos (distancias)

t = {} # Tiempo en minutos
for arc in arcos:
    point1 = df.loc[arc[0], ['X COORD.', 'Y COORD.']].values
    point2 = df.loc[arc[1], ['X COORD.', 'Y COORD.']].values
    distance = euclidean(point1, point2)
    t[arc] = distance

    
T = t[0, len(df)-2]*2+1 # Time limit
    
    
# Parameter that accounts for the population of node j
l = {(i,j): int(df['POPULATION NO.'][j]) for (i,j) in arcos}  #df['POPULATION NO.'][j]

# Probability of vulnerability of node j
#P_ij = {(i,j): t[0,j]/max(t.values()) if j != len(df)-1 else 0 for (i,j) in arcos}
P_ij = {(i,j): float(df['SURVIVAL_PROBABILITY'][j]) if j != len(df)-1 else 0 for (i,j) in arcos}



G = nx.DiGraph()
G.add_edges_from(arcos)

Auxiliary problem maximising only on fairness

In [ ]:
# Calculates auxiliary's problem cost 
c_q = {(i,j): P_ij[i,j] for (i,j) in arcos}

m = Model('AuxProblem')


# Variables

b = {(i,j): m.addVar(vtype = GRB.BINARY, obj = c_q[i,j], name = 'b' + str(i)+str(j)) for (i,j) in arcos}



start_node = 0
end_node = n+1


# Constraints

# Exit origin node only once
m.addConstr(quicksum(b[start_node, j] for j in N) == 1)

# Enter destination node only once
m.addConstr(quicksum(b[i, end_node] for i in N) == 1)

# Flow constraints
for i in N:
    m.addConstr(quicksum(b[i,j] for j in G.successors(i)) - quicksum(b[j,i] for j in G.predecessors(i)) == 0)


### Chat GPT


# Add binary variable for each node to track whether it is visited in the path
z = {}
for i in G.nodes():
    z[i] = m.addVar(vtype=GRB.BINARY, name='z_{0}'.format(i))    
    
# Add MTZ constraints for each node (except start and end)
M = G.number_of_nodes() + 1
u = {}
for i in G.nodes():
    if i != start_node and i != end_node:
        u[i] = m.addVar(vtype=GRB.CONTINUOUS, name='u_{0}'.format(i))

for i in G.nodes():
    if i != start_node and i != end_node:
        for j in G.nodes():
            if j != start_node and j != end_node and i != j:
                expr = u[i] - u[j] + M * b[i, j] <= M - 1 - M * (1 - z[j])
                m.addConstr(expr)


                
# Resource constraint (time)
m.addConstr(quicksum(t[i,j]*b[i,j] for (i,j) in arcos) <= T)



# Maximise
m.ModelSense = -1


# Model's parameters
m.setParam("OutputFlag", 1) 
m.setParam("DualReductions", 0)
m.setParam("InfUnbdInfo", 1)
m.setParam('Threads', 8)
m.setParam('MIPGap', 0.7)


m.update()
m.optimize()

In [ ]:
bvals = { k : v.X for k,v in b.items() }

cOF = sum(bvals[i,j] * l[i,j] for (i,j) in arcos)
POF = sum(bvals[i,j] * P_ij[i,j] for (i,j) in arcos)



min_c_OF = cOF
max_p_OF = POF

print('When only P_ij is considered in the OF, cost:', cOF,'and probability', POF)

In [ ]:
for (i,j) in bvals:
    if bvals[i,j] > 0.5:
        print(i,j)

Auxiliary problem maximising only on node coverage

In [ ]:
# Calculates auxiliary's problem cost 
c_q = {(i,j): l[i,j] for (i,j) in arcos}

m = Model('AuxProblem')


# Variables

b = {(i,j): m.addVar(vtype = GRB.BINARY, obj = c_q[i,j], name = 'b' + str(i)+str(j)) for (i,j) in arcos}


start_node = 0
end_node = n+1


# Constraints

# Exit origin node only once
m.addConstr(quicksum(b[start_node, j] for j in N) == 1)

# Enter destination node only once
m.addConstr(quicksum(b[i, end_node] for i in N) == 1)

# Flow constraints
for i in N:
    m.addConstr(quicksum(b[i,j] for j in G.successors(i)) - quicksum(b[j,i] for j in G.predecessors(i)) == 0)


### Chat GPT


# Add binary variable for each node to track whether it is visited in the path
z = {}
for i in G.nodes():
    z[i] = m.addVar(vtype=GRB.BINARY, name='z_{0}'.format(i))    
    
# Add MTZ constraints for each node (except start and end)
M = G.number_of_nodes() + 1
u = {}
for i in G.nodes():
    if i != start_node and i != end_node:
        u[i] = m.addVar(vtype=GRB.CONTINUOUS, name='u_{0}'.format(i))

for i in G.nodes():
    if i != start_node and i != end_node:
        for j in G.nodes():
            if j != start_node and j != end_node and i != j:
                expr = u[i] - u[j] + M * b[i, j] <= M - 1 - M * (1 - z[j])
                m.addConstr(expr)


                
# Resource constraint (time)
m.addConstr(quicksum(t[i,j]*b[i,j] for (i,j) in arcos) <= T)



# Maximise
m.ModelSense = -1


# Model's parameters
m.setParam("OutputFlag", 1) 
m.setParam("DualReductions", 0)
m.setParam("InfUnbdInfo", 1)
m.setParam('Threads', 8)
m.setParam('MIPGap', 0.2)

m.update()
m.optimize()

In [ ]:
bvals = { k : v.X for k,v in b.items() }

cOF = sum(bvals[i,j] * l[i,j] for (i,j) in arcos)
POF = sum(bvals[i,j] * P_ij[i,j] for (i,j) in arcos)



max_c_OF = cOF
min_p_OF = POF

print('When only c_ij is considered in the OF, cost:', cOF,'and probability', POF)

In [ ]:
for (i,j) in bvals:
    if bvals[i,j] > 0.5:
        print(i,j)

### Pareto front procedure

Iterative process of reoptimising 

In [ ]:
beta = 10e-5
rango = max_p_OF - min_p_OF

OFc = []
OFp = []

OF_p_min = min_p_OF

In [ ]:
# Calculates auxiliary's problem cost using dual variables of the relaxed master problem
c_q = {(i,j): l[i,j] for (i,j) in arcos}
iter_n = 0


while OF_p_min < max_p_OF:
    
    
    iter_n += 1
    m = Model('AuxProblem: '+str(iter_n))


    # Variables

    b = {(i,j): m.addVar(vtype = GRB.BINARY, obj = c_q[i,j], name = 'b' + str(i)+str(j)) for (i,j) in arcos}
    slack = m.addVar(vtype = GRB.CONTINUOUS, obj = beta/rango)


    start_node = 0
    end_node = n+1



    ### New constraint

    delta = 0.001
    eta = OF_p_min + delta
    m.addConstr(quicksum(b[i,j] * P_ij[i,j] for (i,j) in arcos) - slack == eta)




    # Constraints

    # Exit origin node only once
    m.addConstr(quicksum(b[start_node, j] for j in N) == 1)

    # Enter destination node only once
    m.addConstr(quicksum(b[i, end_node] for i in N) == 1)

    # Flow constraints
    for i in N:
        m.addConstr(quicksum(b[i,j] for j in G.successors(i)) - quicksum(b[j,i] for j in G.predecessors(i)) == 0)


    ### Chat GPT


    # Add binary variable for each node to track whether it is visited in the path
    z = {}
    for i in G.nodes():
        z[i] = m.addVar(vtype=GRB.BINARY, name='z_{0}'.format(i))    

    # Add MTZ constraints for each node (except start and end)
    M = G.number_of_nodes() + 1
    u = {}
    for i in G.nodes():
        if i != start_node and i != end_node:
            u[i] = m.addVar(vtype=GRB.CONTINUOUS, name='u_{0}'.format(i))

    for i in G.nodes():
        if i != start_node and i != end_node:
            for j in G.nodes():
                if j != start_node and j != end_node and i != j:
                    expr = u[i] - u[j] + M * b[i, j] <= M - 1 - M * (1 - z[j])
                    m.addConstr(expr)



    # Resource constraint (time)
    m.addConstr(quicksum(t[i,j]*b[i,j] for (i,j) in arcos) <= T)



    # Maximise
    m.ModelSense = -1


    # Model's parameters
    m.setParam("OutputFlag", 0) 
    m.setParam("DualReductions", 0)
    m.setParam("InfUnbdInfo", 1)
    m.setParam('Threads', 8)

    #m.setParam('MIPGap', 0.2)

    m.update()
    m.optimize()
    
    
    
    
    bvals = { k : v.X for k,v in b.items() }

    cOF = sum(bvals[i,j] * l[i,j] for (i,j) in arcos)
    POF = sum(bvals[i,j] * P_ij[i,j] for (i,j) in arcos)

    OFc.append(cOF)
    OFp.append(POF)
    
    OF_p_min = POF
    
    print('OF prob:',OF_p_min, ';slack:', slack.x, ';eta:', eta, ';cost OF:', OFc[-1])

In [ ]:
OFc

In [ ]:
OFp

In [ ]:
import matplotlib.pyplot as plt
plt.plot(OFc, OFp)
plt.ylabel('Fairness')
plt.xlabel('Population covered')
plt.title('Pareto front')
plt.show()